In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
class_labels = {
        0: 'Acne and Rosacea Photos',
    1: 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
    2: 'Atopic Dermatitis Photos',
    3: 'Bullous Disease Photos',
    4: 'Cellulitis Impetigo and other Bacterial Infections',
    5: 'Eczema Photos',
    6: 'Exanthems and Drug Eruptions',
    7: 'Hair Loss Photos Alopecia and other Hair Diseases',
    8: 'Herpes HPV and other STDs Photos',
    9: 'Light Diseases and Disorders of Pigmentation',
    10: 'Lupus and other Connective Tissue diseases',
    11: 'Melanoma Skin Cancer Nevi and Moles',
    12: 'Nail Fungus and other Nail Disease',
    13: 'Poison Ivy Photos and other Contact Dermatitis',
    14: 'Psoriasis pictures Lichen Planus and related diseases',
    15: 'Scabies Lyme Disease and other Infestations and Bites',
    16: 'Seborrheic Keratoses and other Benign Tumors',
    17: 'Systemic Disease',
    18: 'Tinea Ringworm Candidiasis and other Fungal Infections',
    19: 'Urticaria Hives',
    20: 'Vascular Tumors',
    21: 'Vasculitis Photos',
    22: 'Warts Molluscum and other Viral Infections'
    
}


In [3]:
# Define constants
IMAGE_SIZE = (224, 224)  # MobileNetV2 input size
BATCH_SIZE = 64  # Experiment with batch size
NUM_CLASSES = len(class_labels)  # The number of classes in your dataset

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'D:/Hackathon/skin dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 15550 images belonging to 23 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    'D:/Hackathon/skin dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3935 images belonging to 23 classes.


In [6]:
# Create a MobileNetV2 base model with pre-trained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
epochs = 100 # Experiment with the number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    epochs=epochs
)

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
242/242 [==============================] - 430s 2s/step - loss: 2.8420 - accuracy: 0.1837 - val_loss: 2.5398 - val_accuracy: 0.2592
Epoch 2/100
242/242 [==============================] - 2097s 9s/step - loss: 2.5537 - accuracy: 0.2509 - val_loss: 2.4419 - val_accuracy: 0.2928
Epoch 3/100
242/242 [==============================] - 411s 2s/step - loss: 2.4554 - accuracy: 0.2808 - val_loss: 2.3918 - val_accuracy: 0.2989
Epoch 4/100
242/242 [==============================] - 422s 2s/step - loss: 2.3843 - accuracy: 0.2953 - val_loss: 2.3378 - val_accuracy: 0.3097
Epoch 5/100
242/242 [==============================] - 415s 2s/step - loss: 2.3366 - accuracy: 0.3086 - val_loss: 2.3064 - val_accuracy: 0.3176
Epoch 6/100
242/242 [==============================] - 395s 2s/step - loss: 2.2888 - accuracy: 0.3178 - val_loss: 2.2817 - val_accuracy: 0.3263
Epoch 7/100
242/242 [==============================] - 422s 2s/step - loss: 2.2536 - accuracy: 0.3300 - val_loss: 2.2649 - val_accuracy

In [8]:
model.save('skin_disease_classifier_mobilenetv2.h5')

c:\Users\Nirmal\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
model.save('skin_disease_classifier_mobilenetv2.keras')